In [16]:
import torch
from torch.autograd import Function
from torch.autograd import Variable
import torch.nn as nn

Check this [official tutorial](http://pytorch.org/docs/master/notes/extending.html) for more information.

In [2]:
a = torch.randn(3,4)
print(a)


 0.1603 -0.1397 -0.7691  0.1643
-0.2076 -1.9886  0.9985 -1.8432
 1.3439  0.0772  0.3334 -0.6732
[torch.FloatTensor of size 3x4]



In [3]:
a[a.lt(0)] *= 0.5

In [4]:
a


 0.1603 -0.0699 -0.3845  0.1643
-0.1038 -0.9943  0.9985 -0.9216
 1.3439  0.0772  0.3334 -0.3366
[torch.FloatTensor of size 3x4]

In [5]:
r, c = a.size()

Comparing different methods for setting values to zero in tensor.

In [6]:
def for_change(a):
    r,c = a.size()
    for i in range(r):
        for j in range(c):
            if a[i,j] < 0:
                a[i,j] = 0
    return a

In [8]:
%timeit bl = for_change(torch.randn(12,34))

333 µs ± 3.64 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
def gt_change(a):
    a[a.lt(0)] = 0.
    return a

In [10]:
%timeit bl_2 = gt_change(torch.randn(12,34).contiguous())

17 µs ± 50.1 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [11]:
# since first need to define a Function custom op
class my_RELU(Function):
    
    def forward(self, input):
        # use the in-built function to pack the variables
        # for computing the backward pass
        self.save_for_backward(input)
        
        # set the values less than zero to be zero
        # done in style
        input1 = input.clone()
        input1[input1.le(0)] = 0.
        # try to do everything with same variable
        # for speed and efficiency
        return input1
    
    def backward(self, grad_output):
        # use the in-built attribute to recover the saved input variables
        input, = self.saved_tensors
        size = input.size()
        # good practice to initialize the gradient variables
        grad_input = None
        
        # make checks for efficiency
        # they are optional, for more information check the link
        if self.needs_input_grad[0]:
            # computation of gradient with respect to the output
            # stop the gradient for values less than zero
            grad_input = grad_output
            grad_input[input.le(0)] = 0.
            
            #print(type(input), input.size(), type(grad_input), grad_input.size())
        return grad_input

Perform a gradient check to see if the gradients computed above (using calculus) match the numerical gradients.

In [12]:
from torch.autograd import gradcheck

In [13]:
inputs = (Variable(torch.randn(4,3).double(), requires_grad=True),)
test = gradcheck(my_RELU(), inputs, eps=1e-6, atol=1e-4)

print(test)

True


Now define the Module layer so that you can use the custom op as a layer also

In [17]:
class my_RELU(nn.Module):
    def forward(self, input):
        return my_RELU()(input)